### En esta Jupyter Notebook vamos a unir los datasets de reclamos 2018/2019 y de arbolado lineal para generar un nuevo csv

***

**Importacion de librerias**

In [4]:
import numpy as np
import pandas as pd
import re

**Seteamos propiedades de Pandas**

In [5]:
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 100)

**Cargamos Dataset de Reclamos 2018 y 2019, y Arbolado Lineal Publico**

In [6]:
df_19 = pd.read_csv('sistema-unico-de-atencion-ciudadana-2019.csv', index_col=0)
df_18 = pd.read_csv('sistema-unico-de-atencion-ciudadana-2018.csv', index_col=0)
df_arb = pd.read_csv("arbolado-publico-lineal.csv")

**Reseteamos índices y pasamos nombre de columnas a minúsculas**

In [7]:
df_19.reset_index( inplace=True )
df_18.reset_index( inplace=True )

df_19.columns = map(str.lower, df_19.columns)
df_18.columns = map(str.lower, df_18.columns)

**Declaración de funciones para eliminar acentos y mergear dataframes**

In [8]:
def remove_accents( col ):
    return col.str.replace("Á", "A").str.replace("É", "E").str.replace("Í", "I").str.replace("Ó", "O").str.replace("Ú", "U")

def merge_df( df_a, df_b ):
    return pd.merge(df_a, df_b, left_on=['domicilio_calle','domicilio_altura'], right_on=['calle','chapa1'])

**Pasamos a mayúsculas el nombre de las calles en el dataframe de Arbolado lineal**

In [9]:
df_arb.calle = df_arb.calle.apply( lambda x: str.upper(x))

**Concatenamos dataframes de reclamos del 2018 y 2019**

In [10]:
dfs = [ df_18, df_19 ]
total_claims = pd.concat(dfs)

**Removemos acentos del DF resultante**

In [11]:
columnas = [ 'categoria', 'subcategoria', 'concepto' ]
for col in columnas:
    total_claims[ col ] = remove_accents( total_claims[ col ] )
    

In [12]:
df = df_arb

In [13]:
#df[(df['chapa1']== 0) & (df['chapa2'] != 0)]
df.chapa1 = df.apply(lambda x: x.chapa2 if (x.chapa1== 0) & (x.chapa2 != 0) else x.chapa1 ,axis=1)

**Mergeamos DFs de Reclamos y Arboles**

In [14]:
merged_dfs = merge_df( total_claims[ ( total_claims.categoria == 'ARBOLADO Y ESPACIOS VERDES' ) ], df_arb )

In [15]:
merged_dfs.to_csv('reclamos_con_arboles.csv')